## Implementing Mutual TLS Notes

This notebook and repo step through building a demo application that implements mutual TLS. All of the material here comes from the following blog post: ["Implementing Mutual SSL Authentication"](https://blog.cloudboost.io/implementing-mutual-ssl-authentication-fc20ab2392b3). I'm stepping through this tutorial (with modifications) for two reasons:

* To see an implementation of mutual TLS.
* To see how UNIX tools can be used to generate self-signed certificates suitable for use within microservice application boundaries.

For detailed notes on public key cryptography more generally see ["Notes on public key infrastructure"](https://www.kaggle.com/residentmario/notes-on-public-key-infrastructure).

### Generate a client RSA private key

RSA is the original asymmetric cryptography algorithm. These days it has been superseded in web usage by so-called elliptical curve algorithms, which are considered stronger and more secure, but are also more computationally intensive.

We begin by generating an RSA public key and private key pair (in the home directory of this repo). This is done by running the following:

```
openssl genrsa 2048 > client_rsa_key.key
```

### Generate a client cert

PEM is the payload format for the private key, but it needs to be wrapped in additional metadata and binary-encoded for efficiency. The format for this is X.509 and the extention of the output is `.cert`.

```
openssl req -new -x509 -nodes -sha256 -days 365 -key client_rsa_key.key -out client_rsa_key.cert
```

After answering some questions this'll result in a `client_rsa_key.cert` on your machine.

### Generate a client public key

```
chmod client_rsa_key.key 400
ssh-keygen -y -f client_rsa_key.key > client_rsa_key.pub
```

The first line is necessary because `ssh-keygen` will refuse to cooperate if the permissions on the file are not locked.

### Repeat for the server

```
openssl genrsa 2048 > server_rsa_key.key
openssl req -new -x509 -nodes -sha256 -days 365 -key server_rsa_key.key -out server_rsa_key.cert
chmod server_rsa_key.key 400
ssh-keygen -y -f server_rsa_key.key > server_rsa_key.pub
```

### Application code

See [here](https://stackoverflow.com/questions/51522632/how-to-configure-an-ssl-certificate-for-an-application-that-runs-in-express) for example Node.JS application code.

```javascript
var https = require('https');
var fs = require('fs');
var express = require('express');

var options = {
    key: fs.readFileSync('/etc/apache2/ssl/server.key'),
    cert: fs.readFileSync('/etc/apache2/ssl/server.crt'),
    requestCert: false,
    rejectUnauthorized: false
};


var app = express();

var server = https.createServer(options, app).listen(3000, function(){
    console.log("server started at port 3000");
});
```

Basically, get the cert and decide whether or not it is legit based on some kind of mechanism that you control. The most robust way to do this is to use Linux's built-in application store, then verify whether or not this certificate is trusted by the root certificate providers in the built-in.

OpenSSL demonstration:

In [2]:
!openssl verify ../server_rsa_key.cert

C = US, ST = Some-State, O = Internet Widgits Pty Ltd
error 18 at 0 depth lookup: self signed certificate
error ../server_rsa_key.cert: verification failed
